### STEP1: Generate COCO Style Annotation

Here we show a basic usage example using DemoDataset in `data/DemoDataset/`

In [72]:
cd mhp_extension/

/home/qiu/Projects/Self-Correction-Human-Parsing/mhp_extension


In [62]:
!python ./coco_style_annotation_creator/test_human2coco_format.py \
--dataset 'Demo' \
--json_save_dir './data/DemoDataset/msrcnn_finetune_annotations' \
--test_img_dir './data/DemoDataset/global_pic'

### STEP2: Generater Instance Prediciton
Here we provide a finetuned cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv model on CIHP dataset with human instance mask. Download the pretrained weight in `pretrain_model/`.

- [detectron2_maskrcnn_cihp_finetune.pth](https://drive.google.com/file/d/1T797HPC9V1mmw0cDoVOPSF1F_rrTcGPG/view?usp=sharing)


In [73]:
cd ./detectron2/tools/

/home/qiu/Projects/Self-Correction-Human-Parsing/mhp_extension/detectron2/tools


In [64]:
!python finetune_net.py \
--num-gpus 1 \
--config-file ../configs/Misc/demo.yaml \
--eval-only MODEL.WEIGHTS /home/qiu/Downloads/models/detectron2/detectron2_maskrcnn_cihp_finetune.pth TEST.AUG.ENABLED False DATALOADER.NUM_WORKERS 0

(conv1): Conv2d(
            1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
          (conv2_offset): Conv2d(1024, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): DeformConv(
            in_channels=1024, out_channels=1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(1, 1), groups=32, deformable_groups=1, bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
          (conv3): Conv2d(
            1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
        )
        (29): DeformBottleneckBlock(
          (conv1): Conv2d(
            1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False
            (norm): FrozenBatchNorm2d(num_features=1024, eps=1e-05)
          )
          (conv2_offset): Conv2d(1024, 18, kernel_size=(3, 3), stri

!python finetune_net.py \
--num-gpus 1 \
--config-file ../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x_copy.yaml \
--eval-only MODEL.WEIGHTS /home/qiu/Downloads/models/detectron2/detectron2_mask_rcnn_fpn_3x_pre.pkl TEST.AUG.ENABLED False DATALOADER.NUM_WORKERS 0

Crop the original image by prediction bbox

In [74]:
cd ../../

/home/qiu/Projects/Self-Correction-Human-Parsing/mhp_extension


!python make_crop_and_mask_w_mask_nms.py \
--img_dir './data/DemoDataset/global_pic' \ 
--save_dir './data/DemoDataset' \
--img_list './data/DemoDataset/annotations/Demo.json' \
--det_res './data/DemoDataset/detectron2_prediction/inference/instances_predictions.pth'

In [75]:
!python make_crop_and_mask_w_mask_nms.py

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 13.71it/s]


### STEP3: Predict Local and Global Result
Download the pretrained weight in `pretrain_model/`.

- [exp_schp_multi_cihp_global.pth](https://drive.google.com/file/d/1s30hj8zeYj0wuTA5Rek-one-v5uT7kX9/view?usp=sharing)
- [exp_schp_multi_cihp_local.pth](https://drive.google.com/file/d/1dwDrXHkhAe_nYtnSqi548zrjo5mlSPF0/view?usp=sharing)

In [76]:
cd ../

/home/qiu/Projects/Self-Correction-Human-Parsing


In [77]:
!export PYTHONPATH=$PYTHONPATH:/home/qiu/Projects/Self-Correction-Human-Parsing

import os
print(sys.path)

In [78]:
!python mhp_extension/global_local_parsing/global_local_evaluate.py \
--data-dir mhp_extension/data/DemoDataset \
--split-name crop_pic \
--model-restore /home/qiu/Projects/Self-Correction-Human-Parsing/deploy/pascal_abn_checkpoint.pth \
--log-dir mhp_extension/data/DemoDataset \
--save-results

image mean: [0.406, 0.456, 0.485]
image std: [0.225, 0.224, 0.229]
input space:BGR
BGR Transformation
Totoal testing sample numbers: 17
100%|███████████████████████████████████████████| 17/17 [00:02<00:00,  6.94it/s]


In [79]:
!python mhp_extension/global_local_parsing/global_local_evaluate.py \
--data-dir mhp_extension/data/DemoDataset \
--split-name global_pic \
--model-restore /home/qiu/Projects/Self-Correction-Human-Parsing/deploy/pascal_abn_checkpoint.pth \
--log-dir mhp_extension/data/DemoDataset \
--save-results

image mean: [0.406, 0.456, 0.485]
image std: [0.225, 0.224, 0.229]
input space:BGR
BGR Transformation
Totoal testing sample numbers: 4
100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.29it/s]


### STEP4: Fusion Prediciton

In [80]:
!python mhp_extension/logits_fusion.py \
--test_json_path ./mhp_extension/data/DemoDataset/crop.json \
--global_output_dir ./mhp_extension/data/DemoDataset/global_pic_parsing \
--gt_output_dir ./mhp_extension/data/DemoDataset/crop_pic_parsing \
--mask_output_dir ./mhp_extension/data/DemoDataset/crop_mask \
--save_dir ./mhp_extension/data/DemoDataset/mhp_fusion_parsing

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 out of   4 | elapsed:    1.6s remaining:    4.9s
[Parallel(n_jobs=24)]: Done   2 out of   4 | elapsed:    5.0s remaining:    5.0s
[Parallel(n_jobs=24)]: Done   4 out of   4 | elapsed:   12.1s remaining:    0.0s
[Parallel(n_jobs=24)]: Done   4 out of   4 | elapsed:   12.1s finished


### Visualization

from PIL import Image
Image.open('./mhp_extension/demo/demo.jpg')

Image.open('./mhp_extension/demo/demo_instance_human_mask.png')

Image.open('./mhp_extension/demo/demo_global_human_parsing.png')

Image.open('./mhp_extension/demo/demo_multiple_human_parsing.png')